<div style="font-size: 200%; font-weight: bold; color: maroon;">Pandas UDF 1 - 
<span style="color: green;">comparación</span></div>
<!-- Paulo Villegas, v. 1.0, 2021 -->

[Introducing Pandas UDF for PySpark](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html)

Vamos a comparar procesado de distintas formas:
1. Realizado directamente con DataFrames
2. Realizado a través de una UDF antigua, no vectorizada
3. Realizado mediante Pandas UDF

In [ ]:
spark

In [ ]:
import time
import pandas as pd

import pyspark.sql.functions as F
from pyspark.sql.types import FloatType, LongType

# Pandas UDF escalar

una función susceptible de ser usada como Pandas UDF de tipo escalar debe
* aceptar como parámetro(s) objeto(s) de tipo Pandas Serie 
* devolver un objeto de tipo Pandas Serie

In [ ]:
# Define la función que usaremos como UDF

import math
import numpy as np

def process_func1(a: pd.Series, b: pd.Series) -> pd.Series:
    '''multiplica dos números'''
    return a * b


## Creación del DataFrame

In [ ]:
# Crea el DataFrame
df = spark.range(0, 1000000).withColumnRenamed('id', 'c1').withColumn('c2', F.col('c1') + 2)

In [ ]:
df = df.repartition(8)

In [ ]:
df.limit(10).toPandas()

# Prueba 1: multiplicación

## 1.1 DataFrame direct

In [ ]:
df11 = df.withColumn('result', F.col('c1')*F.col('c2'))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r11 = df11.agg(F.sum(F.col('result'))).collect()

elapsed1a = time.process_time() - t1a
elapsed1b = time.perf_counter() - t1b

In [ ]:
print(elapsed1a, elapsed1b)

In [ ]:
r11

## 1.2 UDF estándar

Usamos ahora una UDF *no* vectorizada

In [ ]:
process_udf = F.udf( process_func1, LongType())

In [ ]:
df12 = df.withColumn('result', process_udf(F.col('c1'), F.col('c2')))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r12 = df12.agg(F.sum(F.col('result'))).collect()

elapsed1a = time.process_time() - t1a
elapsed1b = time.perf_counter() - t1b

In [ ]:
print(elapsed1a, elapsed1b)

In [ ]:
r12

## 1.3 Pandas UDF

In [ ]:
process_udf_pandas = F.pandas_udf(process_func1, returnType=LongType())

In [ ]:
df13 = df.withColumn('result', process_udf_pandas(F.col('c1'), F.col('c2')))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r13 = df13.agg(F.sum(F.col('result'))).collect()

elapsed2a = time.process_time() - t1a
elapsed2b = time.perf_counter() - t1b

In [ ]:
print(elapsed2a, elapsed2b)

In [ ]:
r13

# Prueba 2: raíz cuadrada + multiplicación

In [ ]:
def process_func22(a, b):
    '''multiplica un número por la raíz cuadrada de otro'''
    return math.sqrt(a) * b

def process_func23(a: pd.Series, b: pd.Series) -> pd.Series:
    '''multiplica un número por la raíz cuadrada de otro, usando NumPy'''
    return np.sqrt(a) * b

## 2.1 DataFrame direct

In [ ]:
df21 = df.withColumn('result', F.sqrt(F.col('c1'))*F.col('c2'))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r21 = df21.agg(F.sum('result')).collect()

elapsed1a = time.process_time() - t1a
elapsed1b = time.perf_counter() - t1b

In [ ]:
print(elapsed1a, elapsed1b)

In [ ]:
r21

## 2.2 UDF estándar

In [ ]:
process_udf_22 = F.udf( process_func22, FloatType())

In [ ]:
df22 = df.withColumn('result', process_udf_22(F.col('c1'), F.col('c2')))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r22 = df22.agg(F.sum('result')).collect()

elapsed1a = time.process_time() - t1a
elapsed1b = time.perf_counter() - t1b

In [ ]:
print(elapsed1a, elapsed1b)

In [ ]:
r22

## 2.3 Pandas UDF

Probamos a convertir a Pandas UDF dos funciones: `process_func22` y `process_func23`

In [ ]:
process_udf_pandas_23 = F.pandas_udf(process_func22, returnType=FloatType())

In [ ]:
df23 = df.withColumn('result', process_udf_pandas_23(F.col('c1'), F.col('c2')))

In [ ]:
t1a = time.process_time()
t1b = time.perf_counter()

r23 = df23.agg(F.sum('result')).collect()

elapsed2a = time.process_time() - t1a
elapsed2b = time.perf_counter() - t1b

In [ ]:
print(elapsed2a, elapsed2b)

In [ ]:
r23

# Prueba local
Probemos localmente que la función que queremos usar como Pandas UDF implementa el API correcto

In [ ]:
s1 = pd.Series([1, 2, 3, 4])
s2 = pd.Series([4, 5, 6, 7])

In [ ]:
process_func23(s1, s2)